**Дерево рішень**

In [ ]:
import numpy as np


In [ ]:
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0


In [ ]:
def test_split(index, value, dataset):
	left, right = [], []
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right


In [ ]:
def gini_index(groups, classes):
	n_instances = sum([len(group) for group in groups])
	gini = 0.0
	for group in groups:
		size = len(group)
		if size == 0:
			continue
		score = 0
		for values in classes:
			p = (1/size)*([row[-1] for row in group].count(values))
			score += p * p
		gini += (1 - score) * (size / n_instances)
	return gini


In [ ]:
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = np.inf, np.inf, np.inf, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}


In [ ]:
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)


In [ ]:
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)


In [ ]:
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root


In [ ]:
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']


**Будуэмо дерево**

In [ ]:
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)


In [ ]:
num_samples,bias = 100,1.1
theta = np.linspace(0, 2*np.pi, num_samples)
r1,r2 = np.random.rand((num_samples)),np.random.rand((num_samples))
x1, y1 = (r1 * np.cos(theta)).reshape(-1, 1), (r1 * np.sin(theta)).reshape(-1, 1)
x2, y2 = (r2 * np.cos(theta) + bias).reshape(-1, 1), (r2 * np.sin(theta) + bias).reshape(-1, 1)

s1 = np.hstack([x1,y1, np.ones_like(x1)])
s2 = np.hstack([x2,y2, np.zeros_like(x2)])
data = np.vstack([s1,s2])
data= np.hstack([np.ones([data.shape[0],1]),data])


In [ ]:
np.random.shuffle(data)
split_ind = int(np.floor((num_samples*7)/10))
split_ind
train,test = data[split_ind:],data[:split_ind]
(x_train,y_train) = (train[:,:data.shape[1]-1],train[:,-1])
(x_test,y_test) = (test[:,:data.shape[1]-1],test[:,-1])


In [ ]:
max_depth, min_size = 5,10
predictions = decision_tree(train, test, max_depth, min_size)
accuracy_metric(predictions,y_test)


0.9142857142857143